
# Data labeling  
En este notebook lo que haremos es por un lado tomar el dataset que contiene las 10000 laptops descargadas y comprender las columnas que obtuvimos.
Por otro lado lo que haremos es cargar en una columna nueva un path a donde se encuentran ubicadas las imagenes de este laptop. 
Una vez que entendamos cuantas laptops con fotos tenemos y cuantas pudimos etiquetar, separaremos el dataset en 3. Una parte para entrenamiento, otra para validacion y otra para test. Aquellas que quedaron sin etiqueta son las que querremos etiquetar para poder incorporarlas a nuestra database de donde sacaremos las recomendaciones a los clientes.

El etiquetado buscara clasificar las notebooks en 3 tipos:

- Tradicional economica
- Tradicional premium
- Gamers

In [1]:
import pandas as pd
import os
import json
import ast
import numpy as np
from pyprojroot import here

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
path = here() / "data"

In [4]:
df = pd.read_csv(path / "datos_laptops_transformed_cleaned.csv",  sep = ";")


## Etiquetado 

In [80]:
df["Es gamer"].value_counts()

No    509
Sí    110
Name: Es gamer, dtype: int64

In [81]:
df["velocidad_max_procesador"].isna().sum()


392

In [82]:
cond_gamer = (df["Es gamer"] == "Sí")
cond_mac_MSI = df["Marca"].isin(["MSI","Apple"])
cond_ram = df["Capacidad RAM"] > 12
cond_procesador = df["linea_procesador"].isin(["Core i7", "Core i9", "Ryzen 7", "Ryzen 9"])


In [83]:
df["label"] = "Estandar Tradicional"
df.loc[cond_gamer, "label"] = "Gamer"
df.loc[df["Es gamer"].isna(), "label"] = ""
df.loc[cond_mac_MSI, "label"] = "Estandar Premium"
df.loc[cond_ram, "label"] = "Estandar Premium"
df.loc[df["Capacidad RAM"] == -1, "label"] = ""
df.loc[df["linea_procesador"].isna(), "label"] = ""
df.loc[cond_procesador, "label"] = "Estandar Premium"


In [84]:
df.label.value_counts()

Estandar Tradicional    338
Estandar Premium        241
                        105
Gamer                    48
Name: label, dtype: int64

In [85]:
df.head()

,id,price,Resolución de la pantalla,Es gamer,Velocidad máxima del procesador,Capacidad del SSD,Memoria RAM,Velocidad de la memoria RAM,Marca del procesador,Capacidad del SSD,Frecuencia de actualización de la pantalla,Marca,linea_procesador,Capacidad RAM,Medida_RAM,velocidad_max_procesador,medida_vel_procesador,label
0,MLA1129251702,164899.00000,1366 px x 768 px,Sí,NaN,256 GB,12 GB,NaN,AMD,256 GB,None,HP,Ryzen 7,12.00000,GB,NaN,NaN,Estandar Premium
1,MLA1147028688,129499.00000,1920 px x 1080 px,No,4 GHz,256 GB,8 GB,3200 MHz,AMD,256 GB,NaN,HP,Ryzen 5,8.00000,GB,4.00000,GHz,Estandar Tradicional
2,MLA1110784573,66999.00000,1366 px x 768 px,No,NaN,128 GB,4 GB,NaN,Intel,128 GB,60 Hz,Otro,Celeron,4.00000,GB,NaN,NaN,Estandar Tradicional
3,MLA1149134293,138100.00000,1920 px x 1080 px,NaN,3.6 GHz,512 GB,8 GB,2666 MHz,Intel,512 GB,NaN,Lenovo,Core i5,8.00000,GB,3.60000,GHz,
4,MLA1140410405,129999.00000,1920 px x 1080 px,No,4.1 GHz,256 GB,8 GB,NaN,Intel,256 GB,60 Hz,Otro,Core i3,8.00000,GB,4.10000,GHz,Estandar Tradicional


In [86]:
to_label = ( df.label == "")
to_train = ( ( df.label != "") & ( df.label.notna()))

In [87]:
df[to_label].shape

(105, 18)

In [88]:
df[to_train].shape

(627, 18)

In [89]:
df[to_label].to_csv("/content/drive/MyDrive/Austral/Maestria/Redes/Trabajo Final Redes/ML_dataset/to_label.csv", index = False)
df[to_train].to_csv("/content/drive/MyDrive/Austral/Maestria/Redes/Trabajo Final Redes/ML_dataset/to_train.csv", index = False)

In [90]:
labels = df[["id", "label"]]
labels.head()
path_images = "/content/drive/MyDrive/Austral/Maestria/Redes/Trabajo Final Redes/imagenes/MLA1652/"
labels["path"] = path_images + "categ-MLA1652_itemid-" + df.id.copy() + ".jpg"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [91]:
lista = []
for file in os.listdir(path_images):
  lista.append(file)


In [92]:
df_fotos = pd.DataFrame(lista)
df_fotos.columns = ["0"]
df_fotos["path_real"] = path_images + + df_fotos["0"].copy()
del df_fotos["0"]
df_fotos.head()

,path_real
0,/content/drive/MyDrive/Austral/Maestria/Redes/...
1,/content/drive/MyDrive/Austral/Maestria/Redes/...
2,/content/drive/MyDrive/Austral/Maestria/Redes/...
3,/content/drive/MyDrive/Austral/Maestria/Redes/...
4,/content/drive/MyDrive/Austral/Maestria/Redes/...


In [93]:
labels.head()

,id,label,path
0,MLA1129251702,Estandar Premium,/content/drive/MyDrive/Austral/Maestria/Redes/...
1,MLA1147028688,Estandar Tradicional,/content/drive/MyDrive/Austral/Maestria/Redes/...
2,MLA1110784573,Estandar Tradicional,/content/drive/MyDrive/Austral/Maestria/Redes/...
3,MLA1149134293,,/content/drive/MyDrive/Austral/Maestria/Redes/...
4,MLA1140410405,Estandar Tradicional,/content/drive/MyDrive/Austral/Maestria/Redes/...


In [94]:
labels.shape

(732, 3)

In [95]:
labels.path.nunique()

724

In [96]:
df_fotos.shape

(9999, 1)

In [97]:
df_fotos.path_real.nunique()

9999

In [98]:
result = labels.merge(df_fotos,
             how = "left",
             left_on = "path",
             right_on = "path_real")
del result["path"], result["id"]

In [99]:
result.head()

,label,path_real
0,Estandar Premium,NaN
1,Estandar Tradicional,NaN
2,Estandar Tradicional,NaN
3,,NaN
4,Estandar Tradicional,/content/drive/MyDrive/Austral/Maestria/Redes/...


In [100]:
result["path_real"] = result["path_real"].str.split("/").str[10]
result["dummy"] = 1

In [101]:
result.head()

,label,path_real,dummy
0,Estandar Premium,NaN,1
1,Estandar Tradicional,NaN,1
2,Estandar Tradicional,NaN,1
3,,NaN,1
4,Estandar Tradicional,categ-MLA1652_itemid-MLA1140410405.jpg,1


In [102]:
del result["dummy"]



{"Estandar Tradicional" : 1,
               "Estandar Premium" : 2,
               "Gamer" : 3}
               

In [103]:
result["numeric_label"] = np.where(result["label"] == "Estandar Tradicional", 1 ,
                                   np.where(result["label"] == "Estandar Premium" ,2 ,
                                            np.where(result["label"] == "Gamer" ,3 , np.NaN)
                                  ))
                                  
del result["label"]

In [104]:
(result.numeric_label == 1).sum()

338

In [105]:
result.numeric_label.isna().sum()

105

In [106]:
to_label_new = ( (result.numeric_label.isna()) & (result.path_real.notna()) )
to_train_new = ( (result.numeric_label != "") & (result.numeric_label.notna())  & (result.path_real.notna()) & (result.path_real != ""))

In [107]:
result[to_label_new].head()

,path_real,numeric_label
7,categ-MLA1652_itemid-MLA1135443731.jpg,NaN
36,categ-MLA1652_itemid-MLA1143705696.jpg,NaN
82,categ-MLA1652_itemid-MLA1135594941.jpg,NaN
93,categ-MLA1652_itemid-MLA1121132117.jpg,NaN
117,categ-MLA1652_itemid-MLA1101075408.jpg,NaN


In [108]:
result[to_label_new].shape

(29, 2)

In [109]:
result[to_train_new].head()

,path_real,numeric_label
4,categ-MLA1652_itemid-MLA1140410405.jpg,1.00000
8,categ-MLA1652_itemid-MLA1136594401.jpg,1.00000
11,categ-MLA1652_itemid-MLA1138721902.jpg,1.00000
14,categ-MLA1652_itemid-MLA1141953786.jpg,1.00000
22,categ-MLA1652_itemid-MLA1141999385.jpg,1.00000


In [110]:
result[to_train_new].shape

(295, 2)

In [111]:
result[to_train_new].to_csv("/content/drive/MyDrive/Austral/Maestria/Redes/Trabajo Final Redes/to_train.csv", index = False)
result[to_label_new].to_csv(path + "to_label.csv", index = False)